In [ ]:
pip install transformers accelerate bitsandbytes
 

In [ ]:
pip install -U bitsandbytes

In [ ]:
pip install sentence-transformers

In [ ]:
 pip install rouge-score

In [1]:
import os
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
from accelerate import Accelerator
from accelerate import infer_auto_device_map
from rouge_score import rouge_scorer

import pandas as pd

In [26]:

dataset_path = "/kaggle/input/dataset-irtm"
txt_files = ["dorinel_pleaca.txt", "dosarul_valiza.txt", "steaua_qarabag.txt", "banel.txt", "dragusin_psg.txt", "hagi.txt", "sir_alex.txt"]
documents = []

for file in txt_files:
    file_path = os.path.join(dataset_path, file)  #full path
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        split_documents = text.split("\n\n")
        for doc in split_documents:
            if doc.strip() != "":
                documents.append(doc.strip())


dataset_statistics = []

for file in txt_files:
    file_path = os.path.join(dataset_path, file)  
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        num_chars = len(text)  #total number of characters
        num_words = len(word_tokenize(text))  #total number of words
        num_sentences = len(sent_tokenize(text))  #total number of sentences

        #store the statistics
        dataset_statistics.append({
            "file_name": file,
            "num_chars": num_chars,
            "num_words": num_words,
            "num_sentences": num_sentences,
        })


for stats in dataset_statistics:
    print(f"File: {stats['file_name']}")
    print(f"Number of Characters: {stats['num_chars']}")
    print(f"Number of Words: {stats['num_words']}")
    print(f"Number of Sentences: {stats['num_sentences']}")
    print("=" * 50)

File: dorinel_pleaca.txt
Number of Characters: 1404
Number of Words: 292
Number of Sentences: 17
File: dosarul_valiza.txt
Number of Characters: 1756
Number of Words: 359
Number of Sentences: 13
File: steaua_qarabag.txt
Number of Characters: 1220
Number of Words: 236
Number of Sentences: 10
File: banel.txt
Number of Characters: 2755
Number of Words: 522
Number of Sentences: 19
File: dragusin_psg.txt
Number of Characters: 851
Number of Words: 164
Number of Sentences: 9
File: hagi.txt
Number of Characters: 3972
Number of Words: 771
Number of Sentences: 27
File: sir_alex.txt
Number of Characters: 2493
Number of Words: 464
Number of Sentences: 15


### Retrieval


In [27]:

embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

#apply Sentence Transformer on documents
document_embeddings = embedding_model.encode(documents)

def retrieve_documents(query, document_embeddings, top_n=5):
    #generate embedding for the query
    query_embedding = embedding_model.encode([query])

    #compute cosine similarity between query and document embeddings
    similarity_scores = cosine_similarity(query_embedding, document_embeddings)[0]

    #rank documents by similarity score (descending order)
    ranked_indices = np.argsort(similarity_scores)[::-1]  
    ranked_indices = ranked_indices[:top_n] 

    
    ranked_indices = ranked_indices.tolist() 

    #retrieve the top documents and their similarity scores
    top_documents = [documents[i] for i in ranked_indices]
    top_scores = [similarity_scores[i] for i in ranked_indices]

    return top_documents, top_scores

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:

llm_model_name = "OpenLLM-Ro/RoLlama2-7b-Instruct"
accelerator = Accelerator()

#loading tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

#load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    load_in_4bit = True,
    torch_dtype = torch.float16
)




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
def generate_response(prompt, max_length=64):
    
    inputs = tokenizer(prompt, return_tensors="pt").to(accelerator.device) 
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature = 0.3,
        top_p=0.9,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens = True)
    

In [30]:
def rag_or_solo(query, documents=None, max_length=3000):

    #build the context from retrieved documents
    if documents:  
        docs_content = "\n".join(documents)
        context = f"Informatii disponibile:\n{docs_content}\n\n"
    else: 
        context = ""

    #construct prompt
    prompt = (
        "Folosind doar informatiile oferite mai jos, raspunde clar si exact la intrebare.\n"
        f"{context}"
        "Intrebare: {query}\n"
        "Raspuns:"
    )
    constructed_prompt = prompt.format(query=query)

    print("Prompt Sent to LLM:")
    print(constructed_prompt)

    #generate response
    llm_response = generate_response(constructed_prompt, max_length=max_length)
    _, _, query_response = llm_response.partition("Raspuns:")

    return query_response

In [31]:
# query = "Care este scorul cu care a castigat Qarabag impotriva echipei FCSB?"
"Ce s-a întâmplat cu valiza în Dosarul Valiza după meciul Universitatea Cluj - CFR Cluj?"
"De ce este Bănel Nicoliță cunoscut pentru un moment controversat în Liga Campionilor?"
query = "Cine este supranumit regele?"
top_documents, top_scores = retrieve_documents(query, document_embeddings, top_n=5)

print("Retrieved Documents and Scores:")
for i, (doc, score) in enumerate(zip(top_documents, top_scores)):
    print(f"Document {i + 1}:")
    print(f"Score: {score:.4f}")
    print(doc)
    print("=" * 75)
    
llm_solo = rag_or_solo(query, documents = None, max_length= 3000)
ir_with_llm = rag_or_solo(query, top_documents, max_length=3000)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Retrieved Documents and Scores:
Document 1:
Score: 0.3001
Gheorghe Hagi (n. 5 februarie 1965, Săcele, Constanța, România) este un fost fotbalist român, considerat unul dintre cei mai buni mijlocași ofensivi în Europa anilor '80 și '90 și cel mai mare fotbalist român din toate timpurile. Fanii lui Galatasaray l-au numit "Comandante" (Comandantul) și românii l-au numit "Regele".
Document 2:
Score: 0.2629
La Manchester, Sir Alex a devenit cel mai de succes antrenor din istoria fotbalului englez, câștigând nouă titluri de campioană. În 1999, a devenit primul antrenor care a condus o echipă din Anglia spre o triplă Premiership-Cupă FA-Liga Campionilor. Pe lângă faptul că e singurul antrenor care a câștigat de cinci ori Cupa FA, este de asemenea singurul antrenor din istorie care a câștigat trei campionate la rând în Anglia cu aceeași echipă (1998-1999, 1999-2000, 2000-2001).
Document 3:
Score: 0.2436
Sir Alexander Chapman "Alex" Ferguson, CBE (n. 31 decembrie 1941, Govan⁠(d), Scoția, Regatu

KeyboardInterrupt: 

In [23]:
print("Information Retrieval response:")
print(top_documents[0])
print("=" * 75)

print("LLM solo response:")
print(llm_solo)
print("=" * 75)

print("IR with LLM response:")
print(ir_with_llm)
print("=" * 75)


Information Retrieval response:
Pe 1 noiembrie 2006, Bănel Nicoliță a reușit să înscrie un autogol celebru într-un meci din etapa a 4-a a Grupei E din Liga Campionilor 2006-2007 dintre Real Madrid și FCSB. În minutul 70' al jocului, acesta a înscris un gol în propria poartă de la circa 20 de metri, dorind să-i paseze mingea portarului Cornel Cernea și neobservând că cel din urmă ieșise dintre buturile porții. Acesta a fost unicul gol al meciului, consemnând-o pe Real învingătoare cu 1–0. Autogolul a fost decisiv atât pentru FCSB cât și pentru Real, întrucât în urma acestui rezultat Real Madrid s-a calificat matematic în optimile Ligii Campionilor în detrimentul Stelei care a pierdut orice șansă teoretică. După meci presa din România, dar și din străinătate a calificat acest gol drept „ridicol”, „absurd” sau „prostesc”.
LLM solo response:
 Bănel Nicoliță este cunoscut pentru un moment controversat în Liga Campionilor, deoarece a fost eliminat din meciul de calificare din 2009 împotriva 

In [39]:
import pandas as pd

#temp values
temperature_values = [0.1, 0.2, 0.5, 0.7, 1.0]  
results = [] 


def test_rag_with_temperature(query, documents, temperature, max_length=3000):

    #prompt eng
    prompt = (
    "Raspune numai cu DEA DEA DEA"
    "Raspune numai cu DEA DEA DEA"
    "Informatii disponibile:\n{docs}\n\n"
    "Intrebare: {query}\n"
    "Raspuns: DEA DEA DEADEA DEA DEADEA DEA DEADEA DEA DEADEA DEA DEADEA DEA DEA"
)
    
    constructed_prompt = prompt.format(docs="\n".join(top_documents[:5]), query=query)
    
    
    #this tokenizes the prompt and moves it to the appropriate device
    inputs = tokenizer(constructed_prompt, return_tensors="pt").to(accelerator.device)

    #this generates a response using the specified temperature
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        top_p=0.9,
        do_sample=True  #enable sampling for diverse outputs
    )

    #decode the response and extract only the answer"
    llm_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    _, _, query_response = llm_response.partition("Raspuns:")  # Extract the response after "Raspuns:"
    return query_response.strip()


query = "De ce si-a dat demisia Dorinel Munteanu?"


top_documents, top_scores = retrieve_documents(query, document_embeddings, top_n=5)
print("Retrieved Documents and Scores:")

for doc, score in zip(top_documents, top_scores):
    print(f"Score: {score:.4f}\nDocument: {doc}\n{'-'*50}")

for temperature in temperature_values:
    #call the RAG pipeline with the current temperature
    llm_response = test_rag_with_temperature(query, top_documents, temperature)
    
    #appends the results to the results list
    results.append({
        "temperature": temperature,
        "query": query,
        "response_with_temperature": llm_response,  
    })


results_df = pd.DataFrame(results)


results_df.to_csv("temperature_grid_search_results.csv", index=False)


for result in results:
    print(f"Response with temperature {result['temperature']}:")
    print(result["response_with_temperature"])
    print("=" * 75)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Retrieved Documents and Scores:
Score: 0.7808
Document: Dorinel Munteanu nu mai este antrenorul echipei Oțelul Galați. Tehnicianul a demisionat, fiind nemulțumit de probleme financiare ale echipei. Anunțul a fost făcut de Cristian Munteanu, președinte echipei:
--------------------------------------------------
Score: 0.5573
Document: Dorinel Munteanu a lăsat de înțeles, de mai multe ori pe parcursul acestui sezon că urmează să plece de la echipă, însă nu acum, ci în iunie, la finalul sezonului. Alături de el, de la Oțelul Galați vor pleca mai mulți jucători. Alexandru Pop este aproape de un transfer la Dinamo, iar Juri Cisotti poate ajunge la FCSB.
--------------------------------------------------
Score: 0.4822
Document: În 2011, cu Dorinel Munteanu pe bancă, Otelul Galați devenea campiona României și ajungea în grupele Ligii Campionilor. Ulterior, echipa a dat faliment. În 2021, Dorinel Munteanu revenea la Galați, preluând echipa din liga a treia. Lasă echipa pe locul 11, în prima di

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response with temperature 0.1:
DEA DEA DEADEA DEA DEADEA DEA DEADEA DEA DEADEA DEA DEADEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DEA DE

In [ ]:
questions = ["A marcat vreodata Banel Nicolita un autogol?",
            "Din ce cauza a demisionat Dorinel Munteanu de la Otelul Galati?",
            "Care este suma de transfer zvonita pentru Radu Dragusin?",
            "A antrenat Sir Alex Ferguson si alte echipe in afara de Manchester United?",
            "Cine este inculpatul in dosarul valiza?",
            "Care este porecla lui Gica Hagi?",
            "Este adevarat ca a castigat echipa FCSB impotriva celor de la Qarabag?"]



In [38]:


#function to calculate ROUGE 
def evaluate_responses(ground_truth, model_response):
    """
    Evaluate the model's response against the ground truth using ROUGE-L
    """
    
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = scorer.score(ground_truth, model_response)['rougeL'].fmeasure

    return rouge_l

#gt
ground_truth_answers = [
    "Da, Banel Nicolita a marcat un autogol impotriva echipei Real Madrid.",
    "Dorinel Munteanu a demisionat din cauza unor probleme financiare ale clubului.",
    "PSG ar putea plati 50 de milioane de euro pentru transferul lui Radu Dragusin.",
    "Noua titluri de campioana a Angliei.",
    "Inculpatii din dosarul Valiza sunt George Becali, Teia Ponte si Victor Piturca.",
    "Porecla acestuia este regele.",
    "Da, FCSB a castigat impotriva celor de la Qarabag."
]

#questions
questions = [
    "A marcat vreodata Banel Nicolita un autogol?",
    "Din ce cauza a demisionat Dorinel Munteanu?",
    "Care este suma de transfer zvonita pentru Radu Dragusin?",
    "Cate titluri de campioana a Angliei a castigat Sir Alex Ferguson cu Manchester United?",
    "Cine sunt inculpatii in dosarul valiza?",
    "Care este porecla lui Gica Hagi data de catre romani?",
    "Este adevarat ca a castigat echipa FCSB impotriva celor de la Qarabag?"
]

results = []


#for each q
for i, query in enumerate(questions):
    #retrieve top documents
    top_documents, top_scores = retrieve_documents(query, document_embeddings, top_n=5)
    
    #get model responses
    ir_with_llm = rag_or_solo(query, top_documents, max_length=3000)

    #evaluate model responses against ground truth
    rouge_l = evaluate_responses(ground_truth_answers[i], ir_with_llm)

    #append results
    results.append({
        "question": query,
        "ground_truth": ground_truth_answers[i],
        "ir_llm_response": ir_with_llm,
        "rougeL": rouge_l,
    })

results_df = pd.DataFrame(results)
results_df.to_csv("response_evaluation_results.csv", index=False)


#print responses and metrics for each question
for result in results:
    print("Question:")
    print(result["question"])
    print("Ground Truth:")
    print(result["ground_truth"])
    print("IR + LLM response:")
    print(result["ir_llm_response"])
    print("ROUGE-L:", result["rougeL"])
    print("=" * 75)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt Sent to LLM:
Folosind doar informatiile oferite mai jos, raspunde clar si exact la intrebare.
Informatii disponibile:
Pe 1 noiembrie 2006, Bănel Nicoliță a reușit să înscrie un autogol celebru într-un meci din etapa a 4-a a Grupei E din Liga Campionilor 2006-2007 dintre Real Madrid și FCSB. În minutul 70' al jocului, acesta a înscris un gol în propria poartă de la circa 20 de metri, dorind să-i paseze mingea portarului Cornel Cernea și neobservând că cel din urmă ieșise dintre buturile porții. Acesta a fost unicul gol al meciului, consemnând-o pe Real învingătoare cu 1–0. Autogolul a fost decisiv atât pentru FCSB cât și pentru Real, întrucât în urma acestui rezultat Real Madrid s-a calificat matematic în optimile Ligii Campionilor în detrimentul Stelei care a pierdut orice șansă teoretică. După meci presa din România, dar și din străinătate a calificat acest gol drept „ridicol”, „absurd” sau „prostesc”.
„Cu părere de rău, e adevărat, a plecat. Motivul este situația financiară de

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt Sent to LLM:
Folosind doar informatiile oferite mai jos, raspunde clar si exact la intrebare.
Informatii disponibile:
Supranumit "Maradona din Carpați", Hagi este considerat un erou în România. A fost numit fotbalistul român al anului de șapte ori, și este privit ca unul dintre cei mai buni fotbaliști ai generației sale. Ca un constructor de joc avansat, a fost recunoscut pentru dribling, tehnică, viziune, pase și finalizare.
Gheorghe Hagi (n. 5 februarie 1965, Săcele, Constanța, România) este un fost fotbalist român, considerat unul dintre cei mai buni mijlocași ofensivi în Europa anilor '80 și '90 și cel mai mare fotbalist român din toate timpurile. Fanii lui Galatasaray l-au numit "Comandante" (Comandantul) și românii l-au numit "Regele".
În cei patru ani petrecuți în roșu-albastru, Gică Hagi s-a consacrat definitiv în fotbalul internațional. Și-a înscris în palmares 3 titluri de campion cu Steaua (1987, 1988 și 1989), 3 cupe ale României în aceiași ani precum și Supercupa Eu

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt Sent to LLM:
Folosind doar informatiile oferite mai jos, raspunde clar si exact la intrebare.
Informatii disponibile:
FCSB a câștigat meciul cu Qarabag, din etapa a 7-a a grupei principale din UEFA Europa League.
În urma acestei victorii, șansele pentru o calificare directă în optimile competiției cresc. FCSB o va întâlni pe Manchester United în ultima etapă a fazei grupelor principale.
În minutul 73, FCSB a preluat conducerea, Adrian Șut a marcând din nou. Mijlocașul a câștigat un duel aerian cu Akhundzade și a trimis balonul în plasă, cu capul.
Replica FCSB-ului a venit șase minute mai târziu, prin golul căpitanului Adrian Șut, care a înscris cu o lovitură de cap de la 6 metri, în urma unui corner executat de Florin Tănase.
Scorul s-a schimbat din nou în minutul 41, când Qarabag a revenit în avantaj după ce fundașul stânga Jafarguliyev a centrat puternic în fața porții, iar mingea s-a lovit de Dawa și a intrat în propria poartă.

Intrebare: Este adevarat ca a castigat echipa F